In [1]:
!pip install ir-datasets

In [2]:
!pip install elasticsearch

In [3]:
from datetime import datetime
import csv
from elasticsearch import Elasticsearch, helpers
import ir_datasets
import pandas as pd


dataset = ir_datasets.load("clinicaltrials/2021")

In [4]:
df = pd.DataFrame(dataset.docs_iter())

In [6]:
df["content"] = df['title'].astype(str) +" "+ df["summary"].astype(str) + " " + df['detailed_description'].astype(str) + " " + df['eligibility']

In [7]:
df["content"][0]

'Congenital Adrenal Hyperplasia: Calcium Channels as Therapeutic Targets \n    \n      This study will test the ability of extended release nifedipine (Procardia XL), a blood\r\n      pressure medication, to permit a decrease in the dose of glucocorticoid medication children\r\n      take to treat congenital adrenal hyperplasia (CAH).\r\n    \n   \n    \n      This protocol is designed to assess both acute and chronic effects of the calcium channel\r\n      antagonist, nifedipine, on the hypothalamic-pituitary-adrenal axis in patients with\r\n      congenital adrenal hyperplasia. The multicenter trial is composed of two phases and will\r\n      involve a double-blind, placebo-controlled parallel design. The goal of Phase I is to examine\r\n      the ability of nifedipine vs. placebo to decrease adrenocorticotropic hormone (ACTH) levels,\r\n      as well as to begin to assess the dose-dependency of nifedipine effects. The goal of Phase II\r\n      is to evaluate the long-term effects of

In [8]:
df["content"] = df["content"].replace('\n','', regex=True)
df['content'] = df["content"].replace('\r','', regex=True)

In [9]:
df["content"][0]

'Congenital Adrenal Hyperplasia: Calcium Channels as Therapeutic Targets           This study will test the ability of extended release nifedipine (Procardia XL), a blood      pressure medication, to permit a decrease in the dose of glucocorticoid medication children      take to treat congenital adrenal hyperplasia (CAH).                 This protocol is designed to assess both acute and chronic effects of the calcium channel      antagonist, nifedipine, on the hypothalamic-pituitary-adrenal axis in patients with      congenital adrenal hyperplasia. The multicenter trial is composed of two phases and will      involve a double-blind, placebo-controlled parallel design. The goal of Phase I is to examine      the ability of nifedipine vs. placebo to decrease adrenocorticotropic hormone (ACTH) levels,      as well as to begin to assess the dose-dependency of nifedipine effects. The goal of Phase II      is to evaluate the long-term effects of nifedipine; that is, can attenuation of ACTH 